In [38]:
from pyspark.sql import SparkSession
from operator import add
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("adam-axelsson")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [77]:
swe = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.sv")
eng = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.sv-en.en")

#A.1.1
print("Line count Swedish: %d" % swe.count())
#A.1.2
print("Line count English: %d" % eng.count())

#A.1.3
assert(swe.count() == eng.count())

#A.1.4
totalPartitions = swe.getNumPartitions() + eng.getNumPartitions()
print("Total number of Partitions: %d" % totalPartitions)

Line count Swedish: 1862234
Line count English: 1862234
Total number of Partitions: 5


In [88]:
#A.2.1
def case_insensitive_split(lines):
    lower_case = lines.map(lambda line: line.lower())
    lines_split = lower_case.flatMap(lambda line: line.split(' '))
    lines_stripped = lines_split.flatMap(lambda line: line.strip())
    return lines_split

In [90]:
#A.2.2
swe_split = case_insensitive_split(swe)
eng_split = case_insensitive_split(eng)

#A.2.3
print(swe_split.take(10))
print(eng_split.take(10))

['återupptagande', 'av', 'sessionen', 'jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet']
['resumption', 'of', 'the', 'session', 'i', 'declare', 'resumed', 'the', 'session', 'of']


In [91]:
def word_count(word_list):
    mapper = word_list.map(lambda word: (word, 1))
    reducer = mapper.reduceByKey(add)
    return reducer

In [97]:

#A.3.1
word_count_eng = word_count(eng_split)
word_count_eng_sorted = word_count_eng.sortBy(lambda x: -x[1])

word_count_swe = word_count(swe_split)
word_count_swe_sorted = word_count_swe.sortBy(lambda x: -x[1])

print("Most common english words: ")
print(word_count_eng_sorted.take(10))
print("\n\n")
print("Most common swedish words: ")
print(word_count_swe_sorted.take(10))

"""
Results seem reasonable, according to wikipedia these are the most common english words:
the,
be,
to,
of,
a,
and,
in,
that,
have,
i.
Which is similar to these results

For swedish 
"""

Most common english words: 
[('the', 3498375), ('of', 1659758), ('to', 1539760), ('and', 1288401), ('in', 1085993), ('that', 797516), ('a', 773522), ('is', 758050), ('for', 534242), ('we', 522849)]



Most common swedish words: 
[('att', 1706293), ('och', 1344830), ('i', 1050774), ('det', 924866), ('som', 913276), ('för', 908680), ('av', 738068), ('är', 694381), ('en', 620310), ('vi', 539797)]


In [37]:
spark_session.stop()